In [ ]:
import numpy as np
import pandas as pd
from scipy import spatial
from random import sample
import copy
import re
import nltk
from sklearn.metrics.pairwise import cosine_similarity, 

#import encoding dictionaries. i.e. bert_dict['cat'] = [0.5, -.4, 0.001,...]
bert_dict = np.load('bert.npy', allow_pickle = True); bert_dict = bert_dict[()]
glove_dict = np.load('glove.npy', allow_pickle = True); glove_dict = glove_dict[()]
'''        
#glove embeddings for now, will delete later        
glove_embeddings = {}
with open("./top_50000.txt", 'r', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        tag = nltk.pos_tag([word])[0][1]
        if tag in ['NN', 'NNP'] and len(word) != 1 and word.isalpha():
            vector = np.asarray(values[1:], "float32")
            glove_embeddings[word] = vector
np.save('glove.npy', glove_embeddings)
'''

In [ ]:
#import list of codenames words
codenames_words = []
with open("./codenames_words.txt", 'r') as f:
    for line in f:
        codenames_words.append(line.strip())

In [ ]:
#remove codenames words not in embeddings
def removeInvalidCodenames(embeddings, codenames_words):
    removed_words = []
    for item in codenames_words:
        if item not in embeddings:
            removed_words.append(item)
            codenames_words.remove(item)
    print('Words removed from codenames are', removed_words, len(removed_words))
    return codenames_words

codenames_words = removeInvalidCodenames(glove_dict, codenames_words)

In [ ]:
def generate_single_board(codenames_words):
    #randomly selects three words from the codenames board
    board = sample(codenames_words, 3)
    print('The board we will be finding clues for is', board)
    return board
    
def clue_score(embeddings, board, clue):
    #generates a score for a single clue given the board
    if clue in board: #this makes the score for one of the board words really bad
        return -10**6 
    score = 0
    for board_word in board:
        #score += spatial.distance.cosine(embeddings[clue], embeddings[board_word])#1-cosine scoring function
        score += np.linalg.norm(embeddings[clue]-embeddings[board_word])
    return score
    
def clue_dict(embeddings, board):
    #a dictionary for a given board, where the key is the potential clue and the value is the clue score
    score_dict = dict()
    for word in embeddings:
        score_dict[word] = clue_score(embeddings, board, word)
    return score_dict

def extract_top_clues(score_dict):
    sorted_score_dict = sorted(score_dict.items(), key=lambda x:x[1])
    top_ten_clues = [sorted_score_dict[i][0] for i in range(3,3+10)]
    print('Top ten clues are', top_ten_clues)

board = generate_single_board(codenames_words)
score_dict_bert = clue_dict(bert_dict, board)
score_dict_glove = clue_dict(glove_dict, board)
sorted_score_dict = extract_top_clues(score_dict_bert)
sorted_score_dict = extract_top_clues(score_dict_glove)

In [ ]:
print(np.linalg.norm(bert_dict['belt']-bert_dict['kangaroo']))
print(np.linalg.norm(bert_dict['belt']-bert_dict['hotel']))
print(np.linalg.norm(bert_dict['kangaroo']-bert_dict['hotel']))

In [ ]:
print(spatial.distance.cosine(glove_dict['belt'],glove_dict['kangaroo']))
print(spatial.distance.cosine(glove_dict['belt'],glove_dict['hotel']))
print(spatial.distance.cosine(glove_dict['kangaroo'],glove_dict['hotel']))

In [ ]:
def generate_single_board(codenames_words):
    good = sample(codenames_words, 3)

generate_single_example(codenames_words)